# EmojiGen

#### Data Ingestion

In [7]:
# Importing necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Loading emoji descriptions
dataset = pd.read_csv('../data/openmoji.csv')

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   emoji                  4284 non-null   object 
 1   hexcode                4284 non-null   object 
 2   group                  4284 non-null   object 
 3   subgroups              4284 non-null   object 
 4   annotation             4284 non-null   object 
 5   tags                   1907 non-null   object 
 6   openmoji_tags          392 non-null    object 
 7   openmoji_author        4284 non-null   object 
 8   openmoji_date          4284 non-null   object 
 9   skintone               1875 non-null   object 
 10  skintone_combination   2198 non-null   object 
 11  skintone_base_emoji    2198 non-null   object 
 12  skintone_base_hexcode  2198 non-null   object 
 13  unicode                3903 non-null   object 
 14  order                  3782 non-null   float64
dtypes: f

In [10]:
dataset.head()

,emoji,hexcode,group,subgroups,annotation,tags,openmoji_tags,openmoji_author,openmoji_date,skintone,skintone_combination,skintone_base_emoji,skintone_base_hexcode,unicode,order
0,😀,1F600,smileys-emotion,face-smiling,grinning face,"face, grin",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,1,1.0
1,😃,1F603,smileys-emotion,face-smiling,grinning face with big eyes,"face, mouth, open, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,2.0
2,😄,1F604,smileys-emotion,face-smiling,grinning face with smiling eyes,"eye, face, mouth, open, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,3.0
3,😁,1F601,smileys-emotion,face-smiling,beaming face with smiling eyes,"eye, face, grin, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,4.0
4,😆,1F606,smileys-emotion,face-smiling,grinning squinting face,"face, laugh, mouth, satisfied, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,5.0


In [11]:
feature_elimination_list = ['group', 'subgroups', 'openmoji_tags', 'openmoji_author', 'openmoji_date', 'unicode', 'order']

dataset_df = dataset.drop(feature_elimination_list, axis=1)

In [12]:
dataset_df.head()

,emoji,hexcode,annotation,tags,skintone,skintone_combination,skintone_base_emoji,skintone_base_hexcode
0,😀,1F600,grinning face,"face, grin",NaN,NaN,NaN,NaN
1,😃,1F603,grinning face with big eyes,"face, mouth, open, smile",NaN,NaN,NaN,NaN
2,😄,1F604,grinning face with smiling eyes,"eye, face, mouth, open, smile",NaN,NaN,NaN,NaN
3,😁,1F601,beaming face with smiling eyes,"eye, face, grin, smile",NaN,NaN,NaN,NaN
4,😆,1F606,grinning squinting face,"face, laugh, mouth, satisfied, smile",NaN,NaN,NaN,NaN
